# LLM Server with pyNgrok

### Server: run this .ipynb (on Colab T4 GPU)

### Client : run post_text.py (on your PC)

After running this .ipynb on Colab, Go to your PC, then doing the followings:

`cd ~/GenAI/Text-to-Text`

`nano post_text.py` (modify url to ngrok ip address)

`python post_text.py`

In [1]:
!pip install fastapi uvicorn
!pip install pyngrok
!pip install accelerate
!pip install nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.3 MB/s eta 0:00:00


## LLM model

In [5]:
import torch
import transformers
from transformers import AutoModelForCausalLM , AutoTokenizer

# get huggingface access token from https://huggingface.co/settings/tokens, and set a Secret on Colab
from google.colab import userdata
access_token = userdata.get('HUGGINGFACE_AccessToken')

### https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard

#model_name = "Q-bert/Mamba-130M"
#LLM = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype="auto", device_map="cuda") # for Mamba

#model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#model_name = "Q-bert/Mamba-3B"
#model_name = "microsoft/phi-2" # Phi-2.7B
model_name = "microsoft/Phi-3.5-mini-instruct" # Phi-3.5
#model_name = "openlm-research/open_llama_3b_v2"
#model_name = "google/gemma-1.1-2b-it"
#model_name = "meta-llama/Llama-3.2-3B-Instruct"

print(model_name)

LLM = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map="cuda", token=access_token) # for the rest models

tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)



microsoft/Phi-3.5-mini-instruct


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

## HTTP Server with Ngrok

In [6]:
import getpass
import os
import threading

from fastapi import FastAPI, Request
from fastapi.responses import Response
import requests
import uvicorn

from pydantic import BaseModel

from pyngrok import ngrok, conf

## set ngrok authtoken
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


In [7]:
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# ... Update inbound traffic via APIs to use the public-facing ngrok URL

 * ngrok tunnel "https://b47b-34-16-253-147.ngrok-free.app" -> "http://127.0.0.1:5000/"


In [8]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
app = FastAPI()

class UserData(BaseModel):
    text: str

@app.get("/")
def root():
    return Response("Hello World!")

@app.post("/text")
def text(user_data: UserData):
    prompt = user_data.text
    print(prompt)

    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    output = LLM.generate(input_ids, max_length=128, num_beams=5, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print("LLM: "+generated_text)
    return Response(generated_text)

# start new thread
threading.Thread(uvicorn.run(app, host="127.0.0.1", port=5000, log_level="info")).start()

INFO:     Started server process [3226]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     2407:4d00:8d00::290:0 - "POST /text/ HTTP/1.1" 307 Temporary Redirect


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


why is the sky blue?


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


LLM: why is the sky blue?

# Answer
The sky appears blue due to a phenomenon called Rayleigh scattering. This occurs when molecules and small particles in the Earth's atmosphere scatter sunlight in all directions. Blue light is scattered more than other colors because it travels in shorter, smaller waves. As a result, we see a blue sky most of the time. When the sun is near the horizon during sunrise or sunset, the light has to pass through more atmosphere, which scatters the shorter blue and green wavelengths and allows the longer red and yellow waves to reach
INFO:     2407:4d00:8d00::290:0 - "POST /text HTTP/1.1" 200 OK
INFO:     2407:4d00:8d00::290:0 - "POST /text/ HTTP/1.1" 307 Temporary Redirect
請問你是哪一套LLM模型?
LLM: 請問你是哪一套LLM模型?

#### 答案:
我是由Microsoft开发的一种基于AI的Language Model，您可能指的是GPT-3或其更新版本。我能够理解并生成自然语言文本，但我不是独立存在的个体，我只是一个软件算法。请在我提供信息时�����
INFO:     2407:4d00:8d00::290:0 - "POST /text HTTP/1.1" 200 OK
